In [1]:
REFERENCE = 'Reference'
PROJECT = 'Project'

%load_ext tensorboard

from model.model import *
from process_func import *
import os
DATA_DIR = f'datasets/'
map = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Military', 
       'Col': 'Military', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Military', 
       'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
       'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
titles =['Miss', 'Mr', 'Mrs', 'Royal', 'Other', 'Master', 'Military']

# Run for validation

In [2]:
data = Titanic(DATA_DIR, show_head = False)
prepath = 'preprocessed'
try:
    if not os.path.exists(prepath):
        os.makedirs(prepath)
except OSError:
    print('Error Creating Directory...')
data.Preprocess(map, titles, REFERENCE)
data._data.to_csv(prepath + f"/encoded_{REFERENCE}.csv", index = False)
dataset = data.GetXandY()
# Run for validation

Initializing...
Data Loaded.
Data Preprocessing...
Done Preprocessing.
Returned Data Dictionary


In [3]:
# ref_model = ReferenceModel(dataset['x_train'].shape)
# CP_dir_ref = SetCheckpoint(REFERENCE)
# TB_dir_ref = SetLog(REFERENCE)

Model: "ReferenceModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 16)]              0         
                                                                 
 Dense0 (Dense)              (None, 16)                272       
                                                                 
 Dense1 (Dense)              (None, 8)                 136       
                                                                 
 Output (Dense)              (None, 1)                 9         
                                                                 
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________


In [4]:
proj_model = ProjectModel(dataset['x_train'].shape)
CP_dir_proj = SetCheckpoint(PROJECT)
TB_dir_proj = SetLog(PROJECT)
# Run for validation

Model: "ProjectModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 16)]              0         
                                                                 
 Dense0 (Dense)              (None, 32)                544       
                                                                 
 Dense1 (Dense)              (None, 16)                528       
                                                                 
 Dense2 (Dense)              (None, 8)                 136       
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 batch_normalization (BatchN  (None, 8)                32        
 ormalization)                                                   
                                                      

In [5]:
# BATCH_SIZE = 128
# EPOCHS = 100
# callbacks_ref = DefCallbacks(REFERENCE, CP = CP_dir_ref, TB = TB_dir_ref)
# history_ref = ref_model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks_ref, 
#           validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 2)

In [6]:
BATCH_SIZE = 128
EPOCHS = 200
callbacks_proj = DefCallbacks(PROJECT, CP = CP_dir_proj, TB = TB_dir_proj)
history_proj = proj_model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks_proj, 
          validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 2)

Epoch 1/300
6/6 - 2s - loss: 0.7122 - val_loss: 0.6909 - 2s/epoch - 269ms/step
Epoch 2/300
6/6 - 0s - loss: 0.6566 - val_loss: 0.6776 - 82ms/epoch - 14ms/step
Epoch 3/300
6/6 - 0s - loss: 0.6021 - val_loss: 0.6660 - 82ms/epoch - 14ms/step
Epoch 4/300
6/6 - 0s - loss: 0.5787 - val_loss: 0.6557 - 75ms/epoch - 13ms/step
Epoch 5/300
6/6 - 0s - loss: 0.5353 - val_loss: 0.6471 - 74ms/epoch - 12ms/step
Epoch 6/300
6/6 - 0s - loss: 0.5097 - val_loss: 0.6395 - 81ms/epoch - 13ms/step
Epoch 7/300
6/6 - 0s - loss: 0.5091 - val_loss: 0.6319 - 82ms/epoch - 14ms/step
Epoch 8/300
6/6 - 0s - loss: 0.4938 - val_loss: 0.6256 - 75ms/epoch - 12ms/step
Epoch 9/300
6/6 - 0s - loss: 0.4637 - val_loss: 0.6185 - 85ms/epoch - 14ms/step
Epoch 10/300
6/6 - 0s - loss: 0.4743 - val_loss: 0.6120 - 81ms/epoch - 13ms/step
Epoch 11/300
6/6 - 0s - loss: 0.4632 - val_loss: 0.6053 - 78ms/epoch - 13ms/step
Epoch 12/300
6/6 - 0s - loss: 0.4509 - val_loss: 0.5991 - 78ms/epoch - 13ms/step
Epoch 13/300
6/6 - 0s - loss: 0.4638 -

In [9]:
# submission = PerformanceCheck(ref_model, CP_dir_ref, dataset['x_test'], DATA_DIR)
# submission.to_csv(f"submission/submission_{REFERENCE}.csv", index = False)
submission = PerformanceCheck(proj_model, CP_dir_proj, dataset['x_test'], DATA_DIR)
submission.to_csv(f"submission/submission_{PROJECT}.csv", index = False)

3/3 [==============================] - 0s 1ms/step
0    0.082929
1    0.449533
2    0.109556
3    0.145852
4    0.957239
5    0.145202
6    0.445352
7    0.072553
8    0.512490
9    0.042805
dtype: float32
ProjectModel_Accuracy :  87.56 %


In [8]:
# Title (Mr. Miss. ) - Married?
# Social class from Ticket class and Fare